In [13]:
from utils import Encoder
import torch
from datasets import RoboEireanDataModule
import lightning.pytorch as pl

from models import JetNet, SingleShotDetectorLoss, ObjectDetectionTask
from lightning.pytorch.loggers import TensorBoardLogger

In [18]:
LEARNING_RATE = 1e-1
ALPHA = 2.0
NUM_CLASSES = 1
DEFAULT_SCALINGS = torch.tensor(
    [
        [0.06549374, 0.12928654],
        [0.11965626, 0.26605093],
        [0.20708716, 0.38876095],
        [0.31018215, 0.47485098],
        [0.415882, 0.8048184],
        [0.7293086, 0.8216225],
    ]
)
encoder = Encoder(DEFAULT_SCALINGS, NUM_CLASSES)
model = JetNet(NUM_CLASSES, DEFAULT_SCALINGS.shape[0])
loss = SingleShotDetectorLoss(ALPHA)

# data_module = RoboEireanDataModule("data/raw/", encoder, 128)
# data_module.setup("fit")

In [20]:
version = 3
checkpoint_folder = f"new_logs/lightning_logs/version_{version}/checkpoints/"
checkpoint_path = "epoch=0-step=28.ckpt"
checkpoint_path = checkpoint_folder + checkpoint_path
loaded_model = ObjectDetectionTask.load_from_checkpoint(
    checkpoint_path=checkpoint_path, 
    model=model, 
    loss=loss, 
    encoder=encoder, 
    learning_rate=LEARNING_RATE)
loaded_model

ObjectDetectionTask(
  (model): JetNet(
    (input_layer): NormConv2dReLU(
      (batch_normalization): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2d): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=same, bias=False)
      (relu): ReLU()
    )
    (depth_wise_backbone): Sequential(
      (0): DepthWise(
        (batch_normalization): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2d_1): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
        (conv2d_2): Conv2d(16, 24, kernel_size=(1, 1), stride=(1, 1), padding=same)
        (relu): ReLU()
      )
      (1): DepthWise(
        (batch_normalization): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2d_1): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=same, groups=24, bias=False)
        (conv2d_2): Conv2d(24, 16, kernel_size=(1, 1), stride=(

In [21]:
print(loaded_model.learning_rate)

0.1


In [23]:
data_module = RoboEireanDataModule("data/raw/", encoder, 128)
data_module.setup("fit")

In [32]:
batch = next(iter(data_module.val_dataloader()))
test_sample = batch[0]
test_sample

tensor([[[[0.4782, 0.4401, 0.4555,  ..., 0.4563, 0.4688, 0.4641],
          [0.4794, 0.4468, 0.4460,  ..., 0.4390, 0.4403, 0.4765],
          [0.4625, 0.4600, 0.4668,  ..., 0.4582, 0.4505, 0.4713],
          ...,
          [0.3769, 0.3744, 0.3858,  ..., 0.3455, 0.3292, 0.3347],
          [0.3620, 0.3566, 0.3679,  ..., 0.3710, 0.3333, 0.3480],
          [0.3682, 0.3841, 0.3772,  ..., 0.3642, 0.3345, 0.3755]]],


        [[[0.4644, 0.4334, 0.4438,  ..., 0.4596, 0.4667, 0.4630],
          [0.4558, 0.4240, 0.4253,  ..., 0.4453, 0.4516, 0.4711],
          [0.4435, 0.4357, 0.4437,  ..., 0.4519, 0.4479, 0.4654],
          ...,
          [0.2981, 0.3001, 0.3115,  ..., 0.3162, 0.3042, 0.3077],
          [0.2925, 0.2805, 0.2936,  ..., 0.3503, 0.3116, 0.3219],
          [0.2931, 0.3067, 0.3041,  ..., 0.3426, 0.3146, 0.3480]]],


        [[[0.1556, 0.1572, 0.1581,  ..., 0.4518, 0.4606, 0.4526],
          [0.1533, 0.1544, 0.1542,  ..., 0.4290, 0.4348, 0.4540],
          [0.1518, 0.1518, 0.1520,  ..

In [38]:
loaded_model.eval()
y_hat = loaded_model.model(test_sample)
pred_bounding_boxes, pred_logits = y_hat

In [40]:
pred_logits.shape

torch.Size([128, 480, 2])

In [41]:
pred_bounding_boxes.shape

torch.Size([128, 480, 4])